In [1]:
from tqdm import tqdm
import pandas as pd
import os

In [2]:
devsetPath = 'data/devset.json'
datapath = 'data/wiki-pages-text/'

In [4]:
devset = pd.read_json(devsetPath, orient='index')
devset.reset_index(inplace=True)

Helper Functions

In [ ]:
def getEvidenceText(vocab, listofLists):
    if len(listofLists) > 0:
        evidenceText = []
        for subList in listofLists:
            searchString = subList[0] + ' ' + str(subList[1])
            try:
                evidenceText.append(vocab[searchString])
            except KeyError:
                searchString = normalize(searchString)
                evidenceText.append(vocab[searchString])
        return evidenceText
    else:
        return None

Get page text from the data loaded

In [43]:
finalDF = pd.read_pickle('data/lookupDataset.pkl')

Takes about 2-3 minutes, big file

In [45]:
def getPageText(pageTitle):
    return list(finalDF[finalDF['pageTitle'] == pageTitle]['data'].values)[0]

In [46]:
getPageText('Damon_Albarn')

{0: 'Damon Albarn , OBE -LRB- -LSB- ˈdeɪmən_ˈælbɑrn -RSB- born 23 March 1968 -RRB- is an English musician , singer , songwriter , multi-instrumentalist and record producer .\n',
 1: 'He is the lead singer of the British rock band Blur and co-founder , vocalist , instrumentalist and principal songwriter of the virtual band Gorillaz .\n',
 4: 'Raised in Leytonstone , East London and around Colchester , Essex , Albarn attended the Stanway School , where he met Graham Coxon and eventually formed Blur , whose debut album Leisure was released in 1991 to mixed reviews .\n',
 5: "After spending long periods of time touring the US , Albarn 's songwriting became increasingly influenced by British bands from the 1960s .\n",
 6: 'The result of these influences came in the form of Modern Life Is Rubbish -LRB- 1993 -RRB- , Parklife -LRB- 1994 -RRB- and The Great Escape -LRB- 1995 -RRB- .\n',
 7: 'All three albums received critical acclaim while Blur gained mass popularity in the UK , aided by a Brit

Apply stuff

In [ ]:
vocab = loadVocab(filesInDataPath)
devset['evidence_text'] = devset['evidence'].apply(lambda x: getEvidenceText(vocab, x))